<a href="https://colab.research.google.com/github/BTejas001/MWPS_using_SentenceBERT/blob/main/Addition_wp_model_SentenceBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing data and libraries

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('/content/add_wp_data.csv')

In [ ]:
data

,problems,equations
0,john have 3 apples. He buys 2 more apples. How...,3+2
1,sam have 4 apples. He buys 3 more apples. How ...,4+3
2,ram have 5 apples. He buys 5 more apples. How ...,5+5
3,rob have 10 apples. He buys 11 more apples. Ho...,10+11
4,tim have 9 apples. He buys 5 more apples. How ...,9+5
...,...,...
95,A zoo has 14 elephants. They acquire 2 more el...,14+2
96,George has 18 action figures. He buys 9 more a...,18+9
97,A store has 35 hats. They receive 7 more hats....,35+7
98,Mia has 26 toy cars. She buys 4 more toy cars....,26+4


In [ ]:
data.shape

(100, 2)

# Splitting dataset

In [ ]:
from sklearn.model_selection import train_test_split
X = data['problems']
y = data['equations']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Install and importing Sentence BERT

In [ ]:
!pip install sentence-transformers

## Creating model

In [ ]:
from sentence_transformers import SentenceTransformer

# Load Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavio

## Embedding Training and Testing Data

In [ ]:
problem_embeddings_train = model.encode(X_train.tolist())

In [ ]:
problem_embeddings_test = model.encode(X_test.tolist())

In [ ]:
equation_embeddings_train = model.encode(y_train.tolist())
equation_embeddings_test = model.encode(y_test.tolist())

In [ ]:
problem_embeddings_train.shape

(80, 384)

In [ ]:
equation_embeddings_test.shape

(20, 384)

In [ ]:
problem_embeddings_train

array([[ 0.04956351,  0.06541778,  0.00666842, ...,  0.03320094,
         0.03185305,  0.01168208],
       [ 0.02271448, -0.00893462, -0.03088133, ...,  0.03524915,
         0.05485063, -0.01050797],
       [ 0.0493953 , -0.02391582,  0.00195971, ..., -0.06820045,
         0.06066446,  0.00999129],
       ...,
       [ 0.06115631,  0.03939975, -0.04320575, ..., -0.02499283,
         0.0752714 ,  0.00396226],
       [ 0.0768097 ,  0.01982162,  0.01285996, ..., -0.02751694,
        -0.0071606 , -0.01220011],
       [ 0.0258052 ,  0.04632149,  0.00507278, ..., -0.01914231,
         0.04133967,  0.00032006]], dtype=float32)

# Importing Libraries required for RNN

In [ ]:
from keras import Sequential
from keras.layers import Dense,Embedding,SimpleRNN,Flatten

In [ ]:
len(X_train.iloc[0])

104

In [ ]:
len(X_train.iloc[2])

120

## Padding the embeddings

In [ ]:
!pip install keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
problem_embeddings_train = pad_sequences(problem_embeddings_train,padding='post')
problem_embeddings_test = pad_sequences(problem_embeddings_test,padding='post')
equation_embeddings_test = pad_sequences(equation_embeddings_test,padding='post')
equation_embeddings_train = pad_sequences(equation_embeddings_train,padding='post')

In [ ]:
len(problem_embeddings_train[0])

384

In [ ]:
len(problem_embeddings_train[2])

384

# Creating RNN model

In [ ]:
model = Sequential()
model.add(SimpleRNN(32,input_shape=(384,1),return_sequences=False))
model.add(Dense(1))
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)               │ (None, 32)                  │           1,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,121 (4.38 KB)

 Trainable params: 1,121 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
equation_embeddings_train = np.argmax(equation_embeddings_train, axis=1)
equation_embeddings_test = np.argmax(equation_embeddings_test, axis=1)

In [ ]:
model.fit(problem_embeddings_train,equation_embeddings_train,epochs=5,validation_data=(problem_embeddings_test,equation_embeddings_test))

Epoch 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 283ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 161ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 195ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 179ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 185ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [ ]:
#### DO NOT MODIFY THIS CELL...

# model = Sequential()
# model.add(Embedding(input_dim=vocab_size_problems, output_dim=64, input_length=max_len_problems))
# model.add(SimpleRNN(32, return_sequences=False))
# model.add(Dense(vocab_size_equations, activation='softmax'))  # For multi-class classification

# # Compile the model
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
